# Parameter Editing
This notebook gives a quick recipe for how to do calibration or sensitivity analysis with pywatershed. 
It is a design feature that parameters, more specifically the Parameter class, are read-only because 
it should be the case that parameters supplied are used and the code is not opaquely modifying these.

As a consequence, one has to make the Parameter class editable. Below this is accomplished by doing
`the_parameters.to_dd()` which returns a DatasetDict which is editable. One has to know something about
how DatasetDicts are constructed to edit effectively, information can be found in the [documentation](https://pywatershed.readthedocs.io/en/main/api/generated/pywatershed.base.DatasetDict.html#pywatershed.base.DatasetDict). 
The edited DatasetDict can be made a Parameters object again by `Parameters(**param_dict.data)`, as shown below. 

In [ ]:
# auto-format the code in this notebook
%load_ext jupyter_black

In [ ]:
import pathlib as pl
from pprint import pprint

import numpy as np
import pywatershed as pws
import xarray as xr

In [ ]:
domain_dir = pws.constants.__pywatershed_root__ / "data/drb_2yr"
nb_output_dir = pl.Path("./param_edits")
nb_output_dir.mkdir(exist_ok=True)

In [ ]:
# A legacy PRMS parameter file
params = pws.parameters.PrmsParameters.load(domain_dir / "myparam.param")

In [ ]:
param_list = []
param_files = []
for ii in range(11):
    param_dict = params.to_dd()  # copies by default
    multiplier = ii * 0.05 + 0.75
    print("multiplier = ", multiplier)
    param_dict.data_vars["K_coef"] *= multiplier
    param_file_name = nb_output_dir / f"perturbed_params_{str(ii).zfill(3)}.nc"
    param_files += [param_file_name]
    # These could avoid export to netcdf4 if just using in memory
    # could store in a list like: param_list.append(pws.Parameters(**param_dict.data))
    pws.Parameters(**param_dict.data).to_netcdf(
        param_file_name, use_xr=True
    )  # using xarray, more work necessary for nc4 export

In [ ]:
# this provides a check that the values from file are what we expect
for ff in param_files:
    # the problem arises on the read with xarray default decoding
    # but we can just open the netcdf file as Parameters
    # ds = xr.open_dataset(ff, decode_times=False, decode_timedelta=False)
    # k_coef = ds["K_coef"]
    new_params = pws.Parameters.from_netcdf(ff)
    k_coef = new_params.data_vars["K_coef"]
    multipliers = k_coef / params.data_vars["K_coef"]
    assert (multipliers - multipliers[0] < 1e-15).all()
    print(multipliers[0])